<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/%EC%9D%BC%EB%B3%84%EC%88%98%EC%9D%B5%EB%A5%A0_10_%EC%9D%B4%EC%83%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install finance-datareader

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import datetime
import numpy as np


In [3]:
KS = fdr.StockListing('KOSPI') # KOSPI: 940 종목
KDQ = fdr.StockListing('KOSDAQ') # KOSDAQ: 1,597 종목
KS200=fdr.SnapDataReader('KRX/INDEX/STOCK/1028')

In [4]:
#stock_list=pd.concat([KS, KDQ], axis=0)

In [5]:
codes=KS200['Code'].tolist()

In [ ]:
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 12, 31)
df1 = fdr.DataReader('KS200', start_date, end_date)
df1['Change']

In [35]:
stock_data = {}

# 종가 데이터를 불러오는 for 루프
for code in codes:
    # 각 종목의 2024년부터 현재까지의 주가 데이터 불러오기
    df = fdr.DataReader(code, start_date, end_date)

    # 'Close' 열만 선택하여 종가 데이터를 딕셔너리에 저장
    stock_data[code] = df['Close']

# 모든 종목의 종가 데이터를 하나의 DataFrame으로 결합
result = pd.DataFrame(stock_data)
result

,005930,000660,373220,207940,005380,068270,000270,105560,055550,005490,...,001680,178920,008730,000670,006650,032350,009900,006110,016380,114090
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-02,55500,75700,446000,827000,157000,160000,61500,47600,34300,272000,...,21450,28900,36150,584000,165000,13550,14450,40500,7150,18750
2023-01-03,55400,75600,440500,803000,159000,159000,62400,49050,35000,272000,...,21150,29600,36550,583000,173000,13800,14800,41950,7260,18900
2023-01-04,57800,81000,443000,794000,160500,160500,62000,50500,36400,274500,...,20900,28900,37050,613000,165000,14000,14700,41800,7310,18650
2023-01-05,58200,81400,433500,808000,159000,163000,61100,53900,39450,278500,...,20950,27850,37150,610000,166500,14550,14600,39600,7440,19250
2023-01-06,59000,83100,444000,808000,159500,163500,61500,56700,40600,286000,...,20950,28250,37150,610000,165000,14150,14800,38600,7520,18900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21,75000,140500,423000,712000,198700,187700,95400,53100,39300,486500,...,20950,30900,32650,511000,156900,9260,18890,111300,8890,13430
2023-12-22,75900,140600,419500,720000,199000,185500,95600,52500,38800,480500,...,21000,30400,32000,513000,155300,9280,18810,106500,8870,13560
2023-12-26,76600,140900,415500,722000,198800,188800,96300,52500,39100,473500,...,21200,30450,33000,517000,154100,9220,18600,104300,8960,13430


In [64]:
pct_change = result.pct_change(axis=0)
binary_pct_change = (pct_change > 0.1).astype(int)

In [66]:
dates = pd.date_range(start=pd.Timestamp(start_date), periods=176, freq='D')
# 각 종목별로 처음 1이 나타나는 날짜로부터 120일 동안 ffill 적용
for column in binary_pct_change.columns:
    # 각 종목별로 처음 1이 나타나는 날짜를 찾습니다.
    index_of_ones = binary_pct_change[binary_pct_change[column] == 1].index
    if not index_of_ones.empty:
        start_index = index_of_ones[0]
        end_index = min(start_index + pd.DateOffset(days=176), binary_pct_change.index[-1])

        # start_index로부터 end_index까지 범위 내에서만 ffill을 적용합니다.
        binary_pct_change.loc[start_index:end_index, column] = binary_pct_change.loc[start_index:end_index, column].replace(0, float('nan')).ffill()

# NaN을 적절히 채우고, 정수형으로 변환합니다.
binary_pct_change = binary_pct_change.fillna(0).astype(int)


In [72]:
#binary_pct_change_acc.to_csv('binary_pct_change_acc.csv')

In [74]:
binary_pct_change_acc=binary_pct_change*pct_change

binary_pct_change_acc = pd.concat([binary_pct_change_acc, df1['Change'].rename('BM')], axis=1)

binary_pct_change_acc=binary_pct_change_acc[binary_pct_change_acc != 0].dropna(axis=1, how='all')

non_zero_codes = binary_pct_change_acc.columns

In [89]:
binary_pct_change_acc.fillna(0, inplace=True)  # 예시: 결측값을 0으로 대체


In [103]:
# 수익률 계산 함수 정의
def calculate_cumulative_return(series, days):
    if len(series) < days:
        return None
    return (series.iloc[-days:] + 1).prod() - 1

# 절대 수익률 계산
absolute_returns_dict = {}
relative_returns_dict = {}

for window in windows:
    absolute_returns_dict[window] = {}
    relative_returns_dict[window] = {}

    for code in binary_pct_change_acc.columns:
        if code == 'BM':
            continue

        cumulative_return = calculate_cumulative_return(binary_pct_change_acc[code], window)
        if cumulative_return is not None:
            absolute_returns_dict[window][code] = cumulative_return
            bm_cumulative_return = calculate_cumulative_return(binary_pct_change_acc['BM'], window)
            if bm_cumulative_return is not None:
                relative_returns_dict[window][code] = cumulative_return - bm_cumulative_return

# DataFrame 생성
absolute_returns_df = pd.DataFrame(absolute_returns_dict).T
relative_returns_df = pd.DataFrame(relative_returns_dict).T

# 결과 출력
print("절대 수익률:")
print(absolute_returns_df)
print("\n상대 수익률:")
print(relative_returns_df)

절대 수익률:
       373220    005490    006400    051910    066570    003670    012450  \
5   -0.011561  0.007056  0.021645 -0.009921  0.000000  0.000000  0.000000   
10   0.042683  0.102649  0.098952  0.084783  0.000000  0.000000  0.000000   
30  -0.053156  0.053797  0.052397  0.039583  0.008435  0.108280  0.001556   
60   0.063433 -0.066355  0.039648  0.059448  0.066402  0.600276  0.230402   
120  0.063433  0.004020  0.039648  0.059448 -0.160686  1.836124 -0.022779   

       259960    011200   042700  ...    039130    001430    178920    008730  \
5   -0.073684 -0.114027  0.00000  ... -0.020677  0.000000  0.000000 -0.021605   
10  -0.084634  0.248724  0.00000  ...  0.001923  0.000000  0.000000 -0.040847   
30  -0.050980  0.233774  0.00000  ...  0.039920  0.000000  0.000000 -0.050898   
60   0.122319  0.302728  0.00000  ...  0.119227  0.000000  0.102662  0.142342   
120  0.122319  0.302728  0.71746  ...  0.085417 -0.104536 -0.115747 -0.023112   

       006650    032350    009900    00611

In [102]:
# 절대 수익률 계산
def calculate_rolling_return(series, window):
    if len(series) < window:
        return None
    return (series[-window:] + 1).prod() - 1

absolute_returns_dict = {}
relative_returns_dict = {}

for window in windows:
    rolling_returns = binary_pct_change_acc.apply(lambda x: calculate_rolling_return(x, window))

    absolute_returns_dict[window] = rolling_returns.to_dict()

    bm_return = absolute_returns_dict[window].get('BM')
    if bm_return is not None:
        relative_returns_dict[window] = {code: ret - bm_return for code, ret in absolute_returns_dict[window].items() if code != 'BM'}

# DataFrame 생성
absolute_returns_df = pd.DataFrame(absolute_returns_dict).T
relative_returns_df = pd.DataFrame(relative_returns_dict).T

# 결과 출력
print("절대 수익률:")
print(absolute_returns_df)
print("\n상대 수익률:")
print(relative_returns_df)

절대 수익률:
       373220    005490    006400    051910    066570    003670    012450  \
5   -0.011561  0.007056  0.021645 -0.009921  0.000000  0.000000  0.000000   
10   0.042683  0.102649  0.098952  0.084783  0.000000  0.000000  0.000000   
30  -0.053156  0.053797  0.052397  0.039583  0.008435  0.108280  0.001556   
60   0.063433 -0.066355  0.039648  0.059448  0.066402  0.600276  0.230402   
120  0.063433  0.004020  0.039648  0.059448 -0.160686  1.836124 -0.022779   

       259960    011200   042700  ...    001430    178920    008730    006650  \
5   -0.073684 -0.114027  0.00000  ...  0.000000  0.000000 -0.021605 -0.043750   
10  -0.084634  0.248724  0.00000  ...  0.000000  0.000000 -0.040847 -0.024857   
30  -0.050980  0.233774  0.00000  ...  0.000000  0.000000 -0.050898 -0.014175   
60   0.122319  0.302728  0.00000  ...  0.000000  0.102662  0.142342  0.069930   
120  0.122319  0.302728  0.71746  ... -0.104536 -0.115747 -0.023112  0.069930   

       032350    009900    006110    01638

In [96]:
filtered_relative_returns_df_T=absolute_returns_df.T
filtered_absolute_returns_df_T=relative_returns_df.T

In [97]:
KS1=KS.copy()
KS1.set_index('Code', inplace=True)

In [98]:
filtered_absolute_returns_df_T['Name'] = KS1.loc[filtered_absolute_returns_df_T.index, 'Name']
filtered_relative_returns_df_T['Name'] = KS1.loc[filtered_relative_returns_df_T.index, 'Name']

In [106]:
stats=filtered_relative_returns_df_T.describe()
stats.loc['mean'] / stats.loc['std']

,0
5,0.087685
10,0.448749
30,0.339790
60,0.386032
120,0.369109
